# Dataset Properties

In [1]:
import pandas as pd
import numpy as np
import os
import re
import copy

In [2]:
base_dir = '../../data'

website_categories = {}

for website in os.listdir(base_dir):
    data = {}
    for category in os.listdir(os.path.join(base_dir, website)):
        data[category] = []
    website_categories[website] = data

In [3]:
dataset_insights = {
    'num_images': copy.deepcopy(website_categories),
    'num_comparisons': copy.deepcopy(website_categories),
    'image_size': copy.deepcopy(website_categories),
}

In [4]:
def sort_key(image_name):
    match = re.match(r'image_(\d+)_(\d+)', image_name)
    if match:
        x = int(match.group(1))  # Extract the first number (x)
        y = int(match.group(2))  # Extract the second number (y)
        return (x, y)  # Return a tuple for sorting

In [6]:
for website in os.listdir(base_dir):
    for category in os.listdir(os.path.join(base_dir, website)):
        df = pd.read_csv(os.path.join(base_dir, website, category, 'image_descriptions.csv'))
        df['article_number'] = [sort_key(file)[0] for file in df['image number'].dropna().tolist()]
        dataset_insights['num_images'][website][category] = df['article_number'].value_counts().astype(int).tolist()
        images = [os.path.join(base_dir, website, category, img) for img in os.listdir(os.path.join(base_dir, website, category)) if img.endswith('.jpg')]
        dataset_insights['image_size'][website][category] = [os.path.getsize(img) for img in images]
        dataset_insights['num_comparisons'][website][category] = len([(i, j) for i in df['article_number'].tolist() for j in df['article_number'].tolist() if i < j])

**Total no. of images**

In [7]:
num_images = 0

for website, categories in dataset_insights['num_images'].items():
    for category, images in categories.items():
        num_images += sum(images)

print(f'Total no. of images: {num_images}')

Total no. of images: 4264


**Average no. of images per article**

In [39]:
avg_num_image = []

for website, categories in dataset_insights['num_images'].items():
    # remove paid (we only have main article image for those) + aggregators
    if website in ['www.nytimes.com', 'news.google.com', 'www.washingtonpost.com', 'www.forbes.com', 'msn.com']:
        continue
    for category, images in categories.items():
        avg_num_image += images

print(f'Average no. of images per article: {np.mean(avg_num_image)}')

Average no. of images per article: 1.7937219730941705


**Average image size**

In [14]:
def get_image_sizes(directory):
    total_size = 0
    image_count = 0

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):
                image_path = os.path.join(root, file)
                total_size += os.path.getsize(image_path)
                image_count += 1

    if image_count == 0:
        return 0  

    avg_size = total_size / image_count
    return avg_size 

directory_path = '../../data'

average_image_size = get_image_sizes(directory_path)
print(f'Average image size across all folders: {average_image_size:} ')


Average image size across all folders: 199203.8625381187 


**Total no. of image pair comparisons**

In [40]:
num_comparisons = 0

for website, categories in dataset_insights['num_comparisons'].items():
    for category, count in categories.items():
        num_comparisons += count

print(f'Total no. of comparisons: {num_comparisons}')

Total no. of comparisons: 41031
